## Setup

In [346]:
import json
from dotenv import load_dotenv
from pydantic import BaseModel, Field
from langchain.agents import Tool, tool, load_tools, initialize_agent, AgentType, AgentExecutor
from langchain.agents.react.base import DocstoreExplorer
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain.agents.format_scratchpad import format_to_openai_function_messages
from langchain.tools import BaseTool, StructuredTool
from langchain.utilities.google_serper import GoogleSerperAPIWrapper
from langchain.docstore import Wikipedia
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain.tools.render import render_text_description, format_tool_to_openai_function
from langchain.schema import HumanMessage, AIMessage
from langchain.schema.agent import AgentFinish, AgentAction
from langchain.memory import ConversationBufferMemory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

In [9]:
load_dotenv()
gpt3_5 = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

## Tools

In [130]:
# langchain has some easy Tool wrappers for some tools
google = GoogleSerperAPIWrapper()
google_tool = Tool.from_function(
  func=google.run,
  name="Google",
  description="useful for when you need search for something you are not sure about"
)

In [127]:
# using decorator
@tool
def get_word_length(word: str) -> int:
  """Returns the length of a word."""
  return len(word)

In [165]:
# support for multi input data structures
class Word(BaseModel):
  word: str = Field()
  strategy: str = Field()
  
def word_length(word: str, strategy: str) -> int:
  return len(word) if strategy == "normal" else len(word) * 2

word_tool = StructuredTool.from_function(
  func=word_length,
  name="Word_Length_Calculator",
  description="useful to calculate the length of a word. use strategy='normal' for normal length, strategy='turbo' for turbo length.",
  args_schema=Word
)

In [63]:
class CoolNameTool(BaseTool):
  name = "cool_name"
  description = "useful to determine if a name is cool"
  
  def _run(self, query: str) -> str:
    return "Yes, this is a cool name" if query == "Brian" else "No, this is not a cool name"

In [53]:
tools = [
  Tool(
    name="Search",
    func=google.run,
    description="useful for when you need information about current events and data"
  ),
  word_tool
]

In [ ]:
# alternate way to create tool from custom function
Tool.from_function(
  func=google.run,
  name="Search",
  description="useful for when you need to search"
)

In [54]:
render_text_description(tools)

'Search: useful for when you need information about current events and data\nWord_Length_Calculator: Returns the length of a word.'

## Agents

Every agent type in `langchain` has different characteristics, but they mainly differ in what prompt they are using and how they determine what tools to use. It will use either `ReAct` from langchain or `OpenAI` to manage tool invocations. We will first look at the available `off-the-shelf agent` options.

### Zero-shot ReAct
Uses the ReAct framework to determine which tool to use based solely on the tool's description. A very general purpose action agent. ONLY supports tools with single string input.

In [55]:
# can load builtin tools from langchain.agents
tools = load_tools(["llm-math"], gpt3_5)
# will create simple AgentExecutor (no prompt or pipeline)
agent = initialize_agent(tools, gpt3_5, AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)
print(agent.agent.llm_chain.prompt.template)
agent.run("What is 25 to the power of 0.43 power?")

Answer the following questions as best you can. You have access to the following tools:

Calculator: Useful for when you need to answer questions about math.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Calculator]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


> Entering new AgentExecutor chain...
{
  "id": "chatcmpl-8UJmtg2hKTyvHDdX3Zwq7S5euRSxD",
  "object": "chat.completion",
  "created": 1702236595,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "I need to calculate 25 to the power of 0.43.\nAction: Calculator\nAction I

'3.991298452658078'

### Structured Input ReAct (Structured Chat)

Just like zero-shot but supports multi-input tools.

In [124]:
tools = [word_tool]
agent = initialize_agent(tools, gpt3_5, AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True)
print(agent.agent.llm_chain.prompt.format(agent_scratchpad="{agent_scratchpad}", input="{input}"))
agent.run("What is the length of the word 'boulder'?")

System: Respond to the human as helpfully and accurately as possible. You have access to the following tools:

Word Length Calculator: Word Length Calculator(word: str, strategy: str) -> int - useful to calculate the length of a word. use strategy='normal' for normal length, strategy='turbo' for turbo length., args: {'word': {'title': 'Word', 'type': 'string'}, 'strategy': {'title': 'Strategy', 'type': 'string'}}

Use a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).

Valid "action" values: "Final Answer" or Word Length Calculator

Provide only ONE action per $JSON_BLOB, as shown:

```
{
  "action": $TOOL_NAME,
  "action_input": $INPUT
}
```

Follow this format:

Question: input question to answer
Thought: consider previous and subsequent steps
Action:
```
$JSON_BLOB
```
Observation: action result
... (repeat Thought/Action/Observation N times)
Thought: I know what to respond
Action:
```
{
  "action": "Final Answer",
  "action_in

"The length of the word 'boulder' is 7."

### OpenAI

This agent will let OpenAI make the decision on what tool to use. The tool can accept one or more inputs. You will notice the prompt is very basic. The tool options along with their inputs will be passed along to OpenAI. This will invoke one tool at a time per response. However, the `AgentType.OPENAI_MULTI_FUNCTIONS` will allow a list of tool invocations to be processed.

In [163]:
tools = [google_tool]
agent = initialize_agent(tools, gpt3_5, AgentType.OPENAI_FUNCTIONS, verbose=True)
print(agent.agent.prompt.format(agent_scratchpad=[], input="{input}"))
agent.run("What is the highest priced stock?")

System: You are a helpful AI assistant.
Human: {input}


> Entering new AgentExecutor chain...
{
  "id": "chatcmpl-8UMThBMgslMQpTWYlDMEWCaM6jyHs",
  "object": "chat.completion",
  "created": 1702246937,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "Google",
          "arguments": "{\n  \"__arg1\": \"highest priced stock\"\n}"
        }
      },
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 68,
    "completion_tokens": 17,
    "total_tokens": 85
  },
  "system_fingerprint": null
}


Invoking: `Google` with `highest priced stock`


If you wonder which company has the highest share price in the world, here is the answer. Berkshire Hathaway, the conglomerate headed by legendary investor Warren Buffett, has the most expensive stock in the world, with shares trading at over $400,000 each.{
  "id": "chatcmpl-8UMT

'The highest priced stock is Berkshire Hathaway, with shares trading at over $400,000 each.'

### Conversational

This agent is similar to other ReAct agents, but this one has a system prompt optimized for conversations. I have included memory, since that is common with coversations.

In [271]:
tools = [CoolNameTool()]
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
agent = initialize_agent(tools, gpt3_5, AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION, verbose=True, memory=memory)
print(agent.agent.llm_chain.prompt.format(agent_scratchpad=[HumanMessage(content="this is the scratchpad")], chat_history=[HumanMessage(content="this is the chat history")], input="{input}"))
agent.run("hello")
agent.run("My name is Brian. Is my name cool?")

System: Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, Assistant is a powerful system that can help with a wide range

'Yes, Brian is a cool name.'

In [272]:
# you can see the history of the conversation from the memory
memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='hello'),
  AIMessage(content='Hello! How can I assist you today?'),
  HumanMessage(content='My name is Brian. Is my name cool?'),
  AIMessage(content='Yes, Brian is a cool name.')]}

### Self-ask with Search

A specialized agent to be used with a search tool. The LLM must not be a chat model but a normal model. The search tool name must be `Intermediate Answer`.

In [291]:
# search tool name must be "Intermediate Answer"
llm = OpenAI(temperature=0)
tools = [Tool.from_function(func=google.run, name="Intermediate Answer", description="useful for when you need to ask with search")]
agent = initialize_agent(tools, llm, AgentType.SELF_ASK_WITH_SEARCH, verbose=True)
print(agent.agent.llm_chain.prompt.template)
agent.run("What is the highest grossing movie of all time?")

Question: Who lived longer, Muhammad Ali or Alan Turing?
Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali

Question: When was the founder of craigslist born?
Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952

Question: Who was the maternal grandfather of George Washington?
Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Washin

'Avatar (2009)'

### ReAct Document Store

Uses Wikipedia to search and retrieve information. Requires the `wikipedia` python package.

In [300]:
llm = OpenAI(temperature=0)
docstore = DocstoreExplorer(Wikipedia())
tools = [
  Tool(
    name="Search",
    func=docstore.search,
    description="useful for when you need to ask with search"
  ),
  Tool(
    name="Lookup",
    func=docstore.lookup,
    description="useful for when you need to ask with lookup"
  )
]
agent = initialize_agent(tools, llm, AgentType.REACT_DOCSTORE, verbose=True)
print(agent.agent.llm_chain.prompt.template)
agent.run("Who is the youngest US president?")



Question: What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?
Thought: I need to search Colorado orogeny, find the area that the eastern sector of the Colorado orogeny extends into, then find the elevation range of the area.
Action: Search[Colorado orogeny]
Observation: The Colorado orogeny was an episode of mountain building (an orogeny) in Colorado and surrounding areas.
Thought: It does not mention the eastern sector. So I need to look up eastern sector.
Action: Lookup[eastern sector]
Observation: (Result 1 / 1) The eastern sector extends into the High Plains and is called the Central Plains orogeny.
Thought: The eastern sector of Colorado orogeny extends into the High Plains. So I need to search High Plains and find its elevation range.
Action: Search[High Plains]
Observation: High Plains refers to one of two distinct land regions
Thought: I need to instead search High Plains (United States).
Action: Search[High Plains (United St

'John F. Kennedy'

### LangChain Expression Language (LCEL)

The agent runtime provided by LangChain is `AgentExecutor`. It does support others runtimes like `Baby AGI` and `Auto GPT`. Up to this point, we have been using predefined agents. Now we will turn to creating our own agents using LCEL. Note, previously we referred to the `AgentExecutor` as `agent`, but here we refer to the `RunnableSequence` or `chain` as the `agent`, which is given to the `AgentExecutor` runtime.

1. Building an agent starts with a LLM:

In [302]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
llm.invoke("Who are you?")

{
  "id": "chatcmpl-8UP7sEDN4p4d5H679NQZx0dDzHuXf",
  "object": "chat.completion",
  "created": 1702257116,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "I am an AI language model developed by OpenAI. I am designed to assist with answering questions and engaging in conversations on a wide range of topics."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 11,
    "completion_tokens": 30,
    "total_tokens": 41
  },
  "system_fingerprint": null
}



AIMessage(content='I am an AI language model developed by OpenAI. I am designed to assist with answering questions and engaging in conversations on a wide range of topics.')

2. Then we have tools, which is the same as we have seen before, but let's define one again:

In [306]:
@tool
def is_number_even(number: int) -> bool:
  """Returns true if number is even."""
  return number % 2 == 0

tools = [is_number_even]

3. Next, we have the prompt. When using OpenAI the prompt is simple, but if using another API the prompt may need more instructions and examples. Let's create a prompt for OpenAI:

In [315]:
prompt = ChatPromptTemplate.from_messages([
  (
    "system",
    "You are a helpful AI assistant."
  ),
  ("user", "{input}"),
  MessagesPlaceholder(variable_name="agent_scratchpad")
])
print(prompt.format(input="{input}", agent_scratchpad=[]))

System: You are a helpful AI assistant.
Human: {input}


4. The next step is to make the LLM aware of the available tools.

In [324]:
openai_tools = [format_tool_to_openai_function(t) for t in tools]
llm_with_tools = llm.bind(functions=openai_tools)
print(json.dumps(openai_tools[0], indent=2))

{
  "name": "is_number_even",
  "description": "is_number_even(number: int) -> bool - Returns true if number is even.",
  "parameters": {
    "title": "is_number_evenSchemaSchema",
    "type": "object",
    "properties": {
      "number": {
        "title": "Number",
        "type": "integer"
      }
    },
    "required": [
      "number"
    ]
  }
}


5. Then we need to create the agent using the variables from the previous steps to create a `RunnableSequence` using LCEL:

In [328]:
agent = (
  {
    "input": lambda x: x["input"],
    "agent_scratchpad": lambda x: format_to_openai_function_messages(x["intermediate_steps"])
  }
  | prompt | llm_with_tools | OpenAIFunctionsAgentOutputParser()
)
print(type(agent))

<class 'langchain_core.runnables.base.RunnableSequence'>


In [327]:
# take a look at the AgentAction response from using the agent
agent.invoke({"input": "Is 4 an even number?", "intermediate_steps": []})

{
  "id": "chatcmpl-8UQSZYEBAJ8XMx52S7Fv8nDuRq6fY",
  "object": "chat.completion",
  "created": 1702262243,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "is_number_even",
          "arguments": "{\n  \"number\": 4\n}"
        }
      },
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 71,
    "completion_tokens": 16,
    "total_tokens": 87
  },
  "system_fingerprint": null
}



AgentActionMessageLog(tool='is_number_even', tool_input={'number': 4}, log="\nInvoking: `is_number_even` with `{'number': 4}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "number": 4\n}', 'name': 'is_number_even'}})])

6. Right now we just have a runnable sequence, but the agent still doesn't have the ability to iteratively talk to the LLM until the question is answered. There are multiple ways to do this. First, we'll look at the manual way which demonstrates how it works, but keep in mind this will be more easily done soon:

In [338]:
user_input = "Is 343565 an even number?"
intermediate_steps = []
while True:
  output: AgentAction = agent.invoke({"input": user_input, "intermediate_steps": intermediate_steps})
  if isinstance(output, AgentFinish):
    final_result = output.return_values["output"]
    break # we have the final answer
  else:
    tool: Tool = {"is_number_even": is_number_even}[output.tool] # we could have simply called the tool directly
    tool_result = tool.run(output.tool_input) # AgentAction knows the tool input
    intermediate_steps.append((output, tool_result))

print(final_result)

{
  "id": "chatcmpl-8UQmwS7dGSASMpcn47rhCbK39El8Q",
  "object": "chat.completion",
  "created": 1702263506,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "is_number_even",
          "arguments": "{\n  \"number\": 343565\n}"
        }
      },
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 72,
    "completion_tokens": 17,
    "total_tokens": 89
  },
  "system_fingerprint": null
}

{
  "id": "chatcmpl-8UQmxIqDKGY9zvCPYG3l7YyVaYuiU",
  "object": "chat.completion",
  "created": 1702263507,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "No, 343565 is not an even number."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 99,
    "completion_tokens": 12,
    "total_tokens": 111
  },
  "syst

We can simply this process by using the `AgentExecutor` (the runtime):

In [340]:
agent_runtime = AgentExecutor(agent=agent, tools=tools, verbose=True) # similar to initialize_agent but we provide the custom agent
agent_runtime.invoke({"input": "Is 4 an even number?"})



> Entering new AgentExecutor chain...
{
  "id": "chatcmpl-8UQqg8pwH1eMLQHheiQts99SD8Msy",
  "object": "chat.completion",
  "created": 1702263738,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "is_number_even",
          "arguments": "{\n  \"number\": 4\n}"
        }
      },
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 71,
    "completion_tokens": 16,
    "total_tokens": 87
  },
  "system_fingerprint": null
}


Invoking: `is_number_even` with `{'number': 4}`


True{
  "id": "chatcmpl-8UQqgfHxHfjHMoJDbavSSf6rKGpW3",
  "object": "chat.completion",
  "created": 1702263738,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Yes, 4 is an even number."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "pro

{'input': 'Is 4 an even number?', 'output': 'Yes, 4 is an even number.'}

7. (Optional) We can also give the agent memory if we want a more conversational approach. This requires us to change the `prompt` and `agent`:

In [343]:
memory_key = "chat_history"
prompt = ChatPromptTemplate.from_messages([
  (
    "system",
    "You are a helpful AI assistant."
  ),
  MessagesPlaceholder(variable_name=memory_key), # this is needed to hold the history of the conversation
  ("user", "{input}"),
  MessagesPlaceholder(variable_name="agent_scratchpad")
])

In [344]:
agent = (
  {
    "input": lambda x: x["input"],
    "agent_scratchpad": lambda x: format_to_openai_function_messages(x["intermediate_steps"]),
    "chat_history": lambda x: x["chat_history"]
  }
  | prompt | llm_with_tools | OpenAIFunctionsAgentOutputParser()
)

In [345]:
agent_runtime = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [349]:
chat_history = []
input1 = "is 34234 an even number?"
input2 = "is 1 an even number?"
result1 = agent_runtime.invoke({"input": input1, "chat_history": chat_history})
chat_history.extend([
  HumanMessage(content=input1),
  AIMessage(content=result1["output"])
])
result2 = agent_runtime.invoke({"input": input2, "chat_history": chat_history})
chat_history.extend([
  HumanMessage(content=input2),
  AIMessage(content=result2["output"])
])



> Entering new AgentExecutor chain...
{
  "id": "chatcmpl-8UR1rd6cLQM7yBQuNY6qgmlhkhFu3",
  "object": "chat.completion",
  "created": 1702264431,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "is_number_even",
          "arguments": "{\n  \"number\": 34234\n}"
        }
      },
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 72,
    "completion_tokens": 17,
    "total_tokens": 89
  },
  "system_fingerprint": null
}


Invoking: `is_number_even` with `{'number': 34234}`


True{
  "id": "chatcmpl-8UR1rwYsjAC6eSaf6o7fSrtqpDfZ5",
  "object": "chat.completion",
  "created": 1702264431,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Yes, 34234 is an even number."
      },
      "finish_reason": "stop"
    }
  ],
  "usage"

In [353]:
for c in chat_history:
  print(type(c), c.content)

<class 'langchain_core.messages.human.HumanMessage'> is 34234 an even number?
<class 'langchain_core.messages.ai.AIMessage'> Yes, 34234 is an even number.
<class 'langchain_core.messages.human.HumanMessage'> is 1 an even number?
<class 'langchain_core.messages.ai.AIMessage'> No, 1 is not an even number.
